# Simple RISC-V Addition Test

This notebook tests the FPGA RISC-V pipelined processor with a simple addition program.

**Memory Architecture:**
- **IMEM (Instruction Memory)** - Separate BRAM for instructions (read-only)
- **DMEM (Data Memory)** - Separate BRAM for data (read/write)

**Important:** Your Vivado block diagram MUST have **TWO separate BRAM controllers**:
1. One connected to the IMEM interface (imem_addr, imem_en, imem_dout)
2. One connected to the DMEM interface (dmem_addr, dmem_en, dmem_we, dmem_din, dmem_dout)

**Test Program:**
- Load two numbers from data memory
- Add them together
- Store the result back to memory

In [ ]:
from pynq import Overlay, MMIO
import time

print("=" * 70)
print("RISC-V Processor - Simple Addition Test")
print("=" * 70)

## Step 1: Load the FPGA Bitstream

In [ ]:
print("\n[1/5] Loading FPGA Bitstream...")

ol = Overlay("design_1_wrapper.bit")

# Find GPIO controller
gpio_key = next(k for k in ol.ip_dict if "gpio" in k.lower())

# Setup GPIO for reset control
gpio_addr = int(ol.ip_dict[gpio_key]["phys_addr"])
gpio_range = int(ol.ip_dict[gpio_key]["addr_range"])
gpio = MMIO(gpio_addr, gpio_range)

# Setup BRAM controllers
# axi_bram_ctrl_0 = Instruction Memory (IMEM)
# axi_bram_ctrl_1 = Data Memory (DMEM)
imem_key = ol.mem_dict['axi_bram_ctrl_0']
dmem_key = ol.mem_dict['axi_bram_ctrl_1']

# Setup IMEM
imem_base = imem_key['phys_addr']
imem_size = imem_key['addr_range']
imem = MMIO(imem_base, imem_size)

# Setup DMEM
dmem_base = dmem_key['phys_addr']
dmem_size = dmem_key['addr_range']
dmem = MMIO(dmem_base, dmem_size)

print(f"✓ GPIO Controller: {gpio_key} @ {hex(gpio_addr)}")
print(f"✓ Instruction BRAM: axi_bram_ctrl_0 @ {hex(imem_base)} (size: {hex(imem_size)})")
print(f"✓ Data BRAM: axi_bram_ctrl_1 @ {hex(dmem_base)} (size: {hex(dmem_size)})")
print("✓ Bitstream loaded successfully!")

## Step 2: Load Instructions (RISC-V Assembly Program)

```assembly
# Simple addition program
lw   x5, 0x2000(x0)    # Load first number from data memory
lw   x6, 0x2004(x0)    # Load second number from data memory
add  x7, x5, x6        # x7 = x5 + x6
sw   x7, 0x2008(x0)    # Store result to data memory
j    0                 # Infinite loop (halt)
```

In [ ]:
print("\n[2/5] Loading Instructions into Instruction Memory...")

# RISC-V machine code for the addition program
instructions = [
    0x20002283,  # lw x5, 0x2000(x0)  - Load first number
    0x20402303,  # lw x6, 0x2004(x0)  - Load second number
    0x006283b3,  # add x7, x5, x6     - Add the two numbers
    0x20702423,  # sw x7, 0x2008(x0)  - Store result
    0x0000006f   # j 0                - Jump to self (halt)
]

# Write instructions to instruction memory (IMEM starting at address 0x0)
for i, instr in enumerate(instructions):
    imem.write(i * 4, instr)
    
print(f"✓ Loaded {len(instructions)} instructions to IMEM (Instruction Memory)")
print("  Addresses: 0x0000 - 0x{:04X}".format((len(instructions) - 1) * 4))

## Step 3: Write Test Data to Data Memory

In [ ]:
print("\n[3/5] Writing test data to data memory...")

# Memory layout
DATA_MEM_BASE = 0x2000  # Data memory starts at 0x2000

# Test values: Let's add 42 + 58 = 100
num1 = 42
num2 = 58
expected_result = num1 + num2

# Write the two numbers to data memory (DMEM)
dmem.write(DATA_MEM_BASE + 0, num1)      # First number at 0x2000
dmem.write(DATA_MEM_BASE + 4, num2)      # Second number at 0x2004
dmem.write(DATA_MEM_BASE + 8, 0)         # Clear result location

print(f"✓ Number 1: {num1} (written to 0x{DATA_MEM_BASE:04X})")
print(f"✓ Number 2: {num2} (written to 0x{DATA_MEM_BASE+4:04X})")
print(f"✓ Expected Result: {expected_result}")

## Step 4: Reset and Run the Processor

In [ ]:
print("\n[4/5] Resetting and running the processor...")

# Assert reset (assuming active low reset on GPIO)
gpio.write(0, 0)
time.sleep(0.01)  # Hold reset for 10ms

# De-assert reset to start execution
gpio.write(0, 1)
print("✓ Processor started")

# Give processor time to execute (5 instructions shouldn't take long)
time.sleep(0.1)
print("✓ Execution complete (waited 100ms)")

## Step 5: Read Back and Verify Results

In [ ]:
print("\n[5/5] Reading results from data memory...")

# Read the result from data memory (DMEM)
result = dmem.read(DATA_MEM_BASE + 8)

# Handle signed/unsigned interpretation (assuming 32-bit signed)
if result & 0x80000000:
    result_signed = result - 0x100000000
else:
    result_signed = result

print(f"\n{'='*70}")
print("RESULTS:")
print(f"{'='*70}")
print(f"  Input 1:        {num1}")
print(f"  Input 2:        {num2}")
print(f"  Expected:       {expected_result}")
print(f"  Actual Result:  {result_signed}")
print(f"  Raw Value:      0x{result:08X}")
print(f"{'='*70}")

# Verify the result
if result_signed == expected_result:
    print("✅ TEST PASSED! Processor correctly computed the sum.")
else:
    print("❌ TEST FAILED! Result does not match expected value.")
    print(f"   Difference: {result_signed - expected_result}")

## Optional: Try Different Numbers

You can modify the values below and re-run to test with different numbers:

In [ ]:
# Test with different numbers
test_cases = [
    (10, 20),
    (100, 200),
    (-5, 15),
    (0, 0),
    (-10, -20)
]

print("\n" + "="*70)
print("Running Multiple Test Cases...")
print("="*70)

all_passed = True

for num1, num2 in test_cases:
    expected = num1 + num2
    
    # Write test data to DMEM
    dmem.write(DATA_MEM_BASE + 0, num1 & 0xFFFFFFFF)
    dmem.write(DATA_MEM_BASE + 4, num2 & 0xFFFFFFFF)
    dmem.write(DATA_MEM_BASE + 8, 0)
    
    # Reset and run
    gpio.write(0, 0)
    time.sleep(0.01)
    gpio.write(0, 1)
    time.sleep(0.1)
    
    # Read result from DMEM
    result = dmem.read(DATA_MEM_BASE + 8)
    if result & 0x80000000:
        result_signed = result - 0x100000000
    else:
        result_signed = result
    
    # Check result
    passed = (result_signed == expected)
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{status}: {num1:6d} + {num2:6d} = {result_signed:6d} (expected: {expected:6d})")
    
    if not passed:
        all_passed = False

print("="*70)
if all_passed:
    print("✅ All tests PASSED!")
else:
    print("❌ Some tests FAILED!")